# importing required packages

In [1]:
from kafka import KafkaConsumer
from time import sleep
import json,requests
import boto3
import s3fs
import datetime



# Creating kafka Consumer

In [9]:
consumer = KafkaConsumer("first_topic",bootstrap_servers=["127.0.0.1:9092"],value_deserializer=lambda x:json.loads(x.decode('utf-8')))

In [10]:
s3 = s3fs.S3FileSystem() # Connecting with AWS s3 bucket

# Collecting cities name according to country and state

In [4]:
def get_cities_list(country_code="IN",state_code="MP"):
    url = "https://country-state-city-search-rest-api.p.rapidapi.com/cities-by-countrycode-and-statecode"

    querystring = {"countrycode":country_code,"statecode":state_code}

    headers = {
        "X-RapidAPI-Key": "698b387090msh296ecfa89cd59b8p152797jsn067d7f958374",
        "X-RapidAPI-Host": "country-state-city-search-rest-api.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    return response.json()

In [5]:
cities = get_cities_list()
city=[]
for c in cities:
   city.append(c['name'])
city

['Agar',
 'Ajaigarh',
 'Akodia',
 'Alampur',
 'Alirajpur',
 'Alot',
 'Amanganj',
 'Amarkantak',
 'Amarpatan',
 'Amarwara',
 'Ambah',
 'Amla',
 'Anjad',
 'Antri',
 'Anuppur',
 'Aron',
 'Ashoknagar',
 'Ashta',
 'Babai',
 'Badarwas',
 'Badnawar',
 'Bag',
 'Bagli',
 'Baihar',
 'Baikunthpur',
 'Bakshwaho',
 'Balaghat',
 'Baldeogarh',
 'Bamna',
 'Bamor Kalan',
 'Bamora',
 'Banda',
 'Barela',
 'Barghat',
 'Bargi',
 'Barhi',
 'Barwani',
 'Basoda',
 'Begamganj',
 'Beohari',
 'Berasia',
 'Betma',
 'Betul',
 'Betul Bazar',
 'Bhabhra',
 'Bhainsdehi',
 'Bhander',
 'Bhanpura',
 'Bhawaniganj',
 'Bhikangaon',
 'Bhind',
 'Bhitarwar',
 'Bhopal',
 'Biaora',
 'Bijawar',
 'Bijrauni',
 'Bodri',
 'Burhanpur',
 'Burhar',
 'Chanderi',
 'Chandia',
 'Chandla',
 'Chhatarpur',
 'Chhindwara',
 'Chichli',
 'Chorhat',
 'Daboh',
 'Dabra',
 'Damoh',
 'Datia',
 'Deori Khas',
 'Depalpur',
 'Dewas',
 'Dhamnod',
 'Dhana',
 'Dhar',
 'Dharampuri',
 'Dindori',
 'Etawa',
 'Gadarwara',
 'Garha Brahman',
 'Garhakota',
 'Gautampu

# Passing city names to dump data into s3 bucket

In [13]:
idx = 0
for c in consumer:
    
    time = datetime.datetime.now()
    if c.value.get('error',-1) == -1:
        with s3.open(f"/weather--project/city_wise_all_data/{city[idx]}/{city[idx]}_{time}.json","w") as f:
            json.dump(c.value,f)
            idx+=1
    else:
            idx+=1
        
    if idx == len(city):
        idx = 0